In [190]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
from dateutil.parser import parse

In [191]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [192]:
dfsnp500 = pd.read_csv("resources/snp500.csv")
dfsnp500.head()

,Date,open,high,low,close,adjclose,volume,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1/3/2000,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/4/2000,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/5/2000,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/6/2000,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/7/2000,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
dfsnp500_clean = dfsnp500.loc[:,['Date','open','high','low','close','adjclose','volume']]

#dfsnp500['DateN'] = dfsnp500['Date'].apply(datetime.strptime('Date', '%Y-%m-%d'))

dfsnp500_clean['clean_date'] = pd.to_datetime(dfsnp500_clean['Date'])
dfsnp500_clean.set_index('clean_date',inplace=True)

In [194]:
dfsnp500_clean.head()

,Date,open,high,low,close,adjclose,volume
clean_date,,,,,,,
2000-01-03,1/3/2000,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1/4/2000,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1/5/2000,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1/6/2000,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1/7/2000,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [195]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [196]:
# Pull Kraken BTC price exchange data
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Loaded BCHARTS/KRAKENUSD from cache


In [197]:
btc_usd_price_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [198]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [199]:
# Pull pricing data for 3 more BTC exchanges
# ADD YALE/SPCOMP
exchanges = ['COINBASE','BITSTAMP','ITBIT']


exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    if exchange == 'SPCOMP':
        exchange_code = 'YALE/{}'.format(exchange)
    else:    
        exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [200]:
exchange_data

{'BITSTAMP':                 Open      High       Low     Close  Volume (BTC)  \
 Date                                                               
 2011-09-13      5.80      6.00      5.65      5.97     58.371382   
 2011-09-14      5.58      5.72      5.52      5.53     61.145984   
 2011-09-15      5.12      5.24      5.00      5.13     80.140795   
 2011-09-16      4.82      4.87      4.80      4.85     39.914007   
 2011-09-17      4.87      4.87      4.87      4.87      0.300000   
 2011-09-18      4.87      4.92      4.81      4.92    119.812800   
 2011-09-19      4.90      4.90      4.90      4.90     20.000000   
 2011-09-20      4.92      5.66      4.92      5.66     89.280711   
 2011-09-21      5.70      5.79      5.66      5.66     17.629322   
 2011-09-22      5.68      5.72      5.68      5.72     43.778422   
 2011-09-23      5.70      5.72      5.70      5.72     20.680037   
 2011-09-24      5.70      5.76      5.68      5.68     90.587066   
 2011-09-25      6.05 

In [201]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [202]:
# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [203]:
# Remove "0" values
btc_usd_datasets.replace(0, np.nan, inplace=True)

In [204]:
btc_usd_datasets.tail()

,BITSTAMP,COINBASE,ITBIT,KRAKEN
Date,,,,
2018-01-13,14233.654008,14202.494478,14256.959157,14443.863557
2018-01-14,13544.440953,13633.659200,13619.433483,13742.376128
2018-01-15,13875.586308,13831.601515,13850.189661,13889.047827
2018-01-16,11800.132584,11738.709333,11898.419508,11784.994745
2018-01-17,10459.667942,10513.544365,NaN,10525.626956


In [205]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [206]:
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

In [207]:
#Pull Stock Market Data
#YALE/SPCOMP

In [208]:
# Yale S&P data
yale_usd_s7p = get_quandl_data('YALE/SPCOMP')

Loaded YALE/SPCOMP from cache


In [209]:
dfsnp500_clean.head()

,Date,open,high,low,close,adjclose,volume
clean_date,,,,,,,
2000-01-03,1/3/2000,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1/4/2000,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1/5/2000,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1/6/2000,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1/7/2000,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [210]:
yale_usd_s7p_price = yale_usd_s7p['S&P Composite']

In [211]:
yale_usd_s7p.columns

Index(['S&P Composite', 'Dividend', 'Earnings', 'CPI', 'Long Interest Rate',
       'Real Price', 'Real Dividend', 'Real Earnings',
       'Cyclically Adjusted PE Ratio'],
      dtype='object')

In [212]:
yale_usd_s7p = yale_usd_s7p.rename(columns={'S&P Composite': 'Weighted Price' })

In [213]:
yale_usd_s7p['20171001':'20171215']

,Weighted Price,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
2017-10-31,2557.00,48.423333,NaN,246.663,2.36,2557.108847,48.425395,NaN,31.150018
2017-11-30,2593.61,48.676667,NaN,246.669,2.35,2593.657315,48.677555,NaN,31.614559


In [214]:
dfsnp500_clean = dfsnp500_clean.rename(columns={'adjclose': 'Weighted Price' })
dfsnp500_clean.head()

,Date,open,high,low,close,Weighted Price,volume
clean_date,,,,,,,
2000-01-03,1/3/2000,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1/4/2000,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1/5/2000,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1/6/2000,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1/7/2000,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [215]:
exchange_data['S&P 500'] = dfsnp500_clean['20120101':'20180115']

In [216]:
yale_usd_s7p['20171129':'20171201']

,Weighted Price,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
2017-11-30,2593.61,48.676667,NaN,246.669,2.35,2593.657315,48.677555,NaN,31.614559


In [217]:
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [218]:
btc_usd_datasets.head()

,BITSTAMP,COINBASE,ITBIT,KRAKEN,S&P 500
2011-09-13,5.929231,NaN,NaN,NaN,NaN
2011-09-14,5.590798,NaN,NaN,NaN,NaN
2011-09-15,5.094272,NaN,NaN,NaN,NaN
2011-09-16,4.854515,NaN,NaN,NaN,NaN
2011-09-17,4.870000,NaN,NaN,NaN,NaN


In [219]:
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets['20170101':'20171230'], 'Bitcoin Price (USD) By Exchange Vs S&P Composite')

In [230]:
exchange_data['KRAKEN']

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,1.315147e+04,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,1.609733e+04,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6.784250e+03,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6.780220e+03,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,1.669857e+04,890.671709
2014-01-12,899.96114,900.93989,833.00001,860.00000,25.429433,2.188088e+04,860.454846
2014-01-13,847.32152,859.99999,815.00000,835.00000,25.869127,2.152984e+04,832.260007
2014-01-14,835.00000,877.29300,805.00000,831.00000,31.662881,2.675628e+04,845.036122
2014-01-15,831.00000,864.00000,828.00000,850.00364,6.707565,5.698139e+03,849.509430


In [232]:
#Start Regression
reg_data = exchange_data['KRAKEN'].loc[:,['Open','High', 'Low', 'Close', 'Volume (BTC)' , 'Volume (Currency)', 'Weighted Price']]

In [243]:
X = reg_data['20171001':'20171215'][['Open','High', 'Low', 'Close',  'Volume (BTC)' , 'Volume (Currency)', 'Weighted Price']]

y = reg_data['20171218':'20171229']["Open"].values.reshape(-1, 1)
print(X.shape, y.shape)

(76, 7) (12, 1)


In [251]:
print(X.describe)

<bound method NDFrame.describe of                Open     High      Low    Close  Volume (BTC)  \
Date                                                           
2017-10-01   4343.9   4391.0   4250.0   4383.7   2721.385774   
2017-10-02   4387.9   4467.5   4365.2   4401.7   3475.870910   
2017-10-03   4401.6   4430.3   4240.0   4309.0   5346.374813   
2017-10-04   4312.9   4352.9   4189.1   4223.6   3742.458703   
2017-10-05   4222.3   4372.0   4154.9   4324.2   3422.408527   
2017-10-06   4319.6   4422.1   4300.1   4363.1   3102.323529   
2017-10-07   4363.1   4440.0   4318.5   4432.0   2320.463852   
2017-10-08   4423.2   4615.1   4407.1   4592.0   4105.517197   
2017-10-09   4594.4   4875.0   4555.0   4772.1   5241.182325   
2017-10-10   4774.9   4919.0   4712.4   4748.5   5679.179929   
2017-10-11   4745.4   4878.0   4710.2   4822.6   3619.274646   
2017-10-12   4824.2   5430.0   4812.2   5429.7   7950.666610   
2017-10-13   5429.8   5854.4   5380.1   5640.0   9222.144339   
2017-1

In [250]:
print(y)

[[ 18920.8]
 [ 18832.8]
 [ 17593.5]
 [ 16217.7]
 [ 15184.3]
 [ 13899.8]
 [ 14569.4]
 [ 13760. ]
 [ 13870. ]
 [ 15813.3]
 [ 15444.3]
 [ 14566.1]]
